# INFO 4271 - Group Project

Issued: June 17, 2025

Due: July 21, 2025

Please submit a link to your code repository (with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [ ]:
%load_ext autoreload
%autoreload 2

from project import Crawler
import cProfile

start_urls = []
with open('seed.txt', 'r') as file:
    for line in file: start_urls.append(line.strip())
start_urls
crawler = Crawler(start_urls, max_workers=200, use_proxies=False, auto_resume=True, path="data")
# snakeviz profiler.prof
# cProfile.run('crawler.run()', 'data/profiler.prof')
crawler.run()

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file (see `queries.txt` for an example) will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [2]:
%load_ext autoreload
%autoreload 2

from project import HTML_FILE
from tqdm import tqdm
import os
import json

def load_data(path: str = "data"):
    html_path = os.path.join(path, HTML_FILE)
    if not os.path.exists(html_path):
        raise FileNotFoundError(f"HTML file not found at {html_path}")

    docs = {}
    with open(html_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines, "Line"): docs.update(json.loads(line.strip()))
    return docs

docs = load_data(path="data")

Line: 100%|██████████| 5190/5190 [00:02<00:00, 2263.85it/s]


In [10]:
from bs4 import BeautifulSoup

def preprocess_html(html: str, seperator: str = ' ') -> str:
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(separator=seperator, strip=True)
    return text.strip()

In [11]:
from project import SiglipStyleModel, ColSentenceModel, MentorModel
import torch

def generate_embeddings(model: SiglipStyleModel | ColSentenceModel | MentorModel, data_dict: dict, batch_size: int = 1) -> dict:
    embedding_dict = {}
    items = list(data_dict.items())
    for i in tqdm(range(0, len(data_dict), batch_size), "Embedding"):
        htmls = [preprocess_html(item[1], ". ")[:10000] for item in items[i:i + batch_size]]
        try:
            embeddings = model.embed(htmls).detach().cpu()
            for j, embedding in enumerate(embeddings):
                embedding_dict[embedding] = items[i + j][0]
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"Error embedding: {e}")
    return embedding_dict

model = ColSentenceModel().load("clip/ColSent/bert-mini/b64_lr1E-06_microsoft/ms_marcov2.1/model.safetensors")
# model = SiglipStyleModel().load(r"project/retriever/model_uploads/bert-mini_b32_marco_v1.safetensors")
# model = ColSentenceModel()
embeddings = generate_embeddings(model, docs)

[nltk_data] Downloading package punkt_tab to /home/jan-
[nltk_data]     malte/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Embedding: 100%|██████████| 4956/4956 [05:20<00:00, 15.47it/s]


In [12]:
import torch

def save_embeddings(embeddings: dict, path: str = "data/embeddings.pkl"):
    torch.save(embeddings, path)

save_embeddings(embeddings, path="data/embeddings.pkl")

In [13]:
import torch

def load_embeddings(path: str = "data/embeddings.pkl") -> dict:
    return torch.load(path)

embedding_dict = load_embeddings(path="data/embeddings.pkl")

In [18]:
import numpy as np
DISCOUNT_FACTOR = 0.7
def retrieve(model: SiglipStyleModel | ColSentenceModel | MentorModel, query: str, index: dict[torch.Tensor, str]):
    domain_level_discount = []
    for url in index.values():
        domain_level = url.count("/") - 2
        if "/en/" in url:
            domain_level =- 2 # ignore en tags in discount calc
        domain_level_discount.append(DISCOUNT_FACTOR**domain_level)
    np.array(domain_level_discount)

    similarities = []
    query_embedding = model.embed(query)
    for embedding, _ in tqdm(list(index.items()), "Similarities"):
        similarity = model.resolve(query_embedding, embedding.cuda(), max_sim=False).squeeze()
        similarities.append(similarity.detach().cpu().numpy())
    vals = np.array(list(zip(index.values(), similarities)))
    vals_weighted = np.array(list(zip(index.values(), np.array(similarities)*domain_level_discount)))
    return vals[np.argsort(similarities)[::-1]], vals_weighted[np.argsort(np.array(similarities)*domain_level_discount)[::-1]]

model = ColSentenceModel().load(r"clip/ColSent/bert-mini/b64_lr1E-06_microsoft/ms_marcov2.1/model.safetensors")
# model = SiglipStyleModel().load(r"project/retriever/model_uploads/bert-mini_b32_marco_v1.safetensors")
res, res_weighted = retrieve(model, "University", embedding_dict)
res[:30]

[nltk_data] Downloading package punkt_tab to /home/jan-
[nltk_data]     malte/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Similarities: 100%|██████████| 4956/4956 [00:00<00:00, 6920.93it/s]


array([['https://www.govdirectory.org/germany/', '0.5746084'],
       ['https://hafedlab.org/2025/07/', '0.5296754'],
       ['https://hafedlab.org/2016/02/', '0.5268632'],
       ['https://hafedlab.org/2020/10/', '0.525901'],
       ['https://hafedlab.org/2015/02/', '0.5253572'],
       ['https://www.lawfin.uni-frankfurt.de/fellows/', '0.52497584'],
       ['https://www.lawfin.uni-frankfurt.de/fellows/?L=0', '0.52497584'],
       ['https://hafedlab.org/2014/10/', '0.523549'],
       ['https://hafedlab.org/2016/07/', '0.5227303'],
       ['https://hafedlab.org/2015/04/', '0.52252406'],
       ['https://hafedlab.org/2017/05/', '0.52189964'],
       ['https://hafedlab.org/2016/05/', '0.52115184'],
       ['https://hafedlab.org/2023/04/', '0.5206082'],
       ['https://hafedlab.org/2017/11/', '0.51897144'],
       ['https://hafedlab.org/2023/11/', '0.51872355'],
       ['http://www.cs.ucla.edu/~palsberg/h-number.html', '0.51866806'],
       ['https://hafedlab.org/2018/01/', '0.5178626'],


In [19]:
res_weighted[:30]

array([['https://www.tuebinger-erbe-lauf.de/en/galerie',
        '1.03809870019251'],
       ['http://learning-systems.org/en/sign_in', '1.0184964355157347'],
       ['https://www.qschina.cn/en/universities/eberhard-karls-universitat-tubingen/postgrad',
        '0.971558690071106'],
       ['https://www.qschina.cn/en/universities/eberhard-karls-universitat-tubingen',
        '0.966626162431678'],
       ['http://dw.de/en/headlines/headlines-en', '0.9664471052130874'],
       ['https://www.qschina.cn/en/universities/eberhard-karls-universitat-tubingen/undergrad',
        '0.9591182275694244'],
       ['https://www.uni-stuttgart.de/en/', '0.9586578120990675'],
       ['https://www.uni-stuttgart.de/en/university/news/all/From-the-Lindau-Nobel-Laureate-Meeting-to-Stuttgart-Chemistry/',
        '0.9522028723541571'],
       ['https://www.qschina.cn/en/universities/eberhard-karls-universitat-tubingen/more',
        '0.9497168112774285'],
       ['https://gerit.org/en/institutiondetail/573469

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 22nd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
